In [1]:
!pip3 install ../../random-forest-mc/

Processing /work/random-forest-mc
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for random-forest-mc: filename=random_forest_mc-1.1.0-py3-none-any.whl size=14847 sha256=2f9d4244409a482f64349b5c55c665a6e9f14185ff79d2f9d0d65d3178829371
  Stored in directory: /root/.cache/pip/wheels/eb/5f/19/4722d7f6bf0d13102d1af6ab98e4b53771da1ed44ce678108e
Successfully built random-forest-mc
  Attempting uninstall: random-forest-mc
    Found existing installation: random-forest-mc 1.1.0
    Uninstalling random-forest-mc-1.1.0:
      Successfully uninstalled random-forest-mc-1.1.0

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [2]:
from random_forest_mc.model import RandomForestMC
from random_forest_mc.utils import LoadDicts
import pandas as pd
import numpy as np

from typing import Any
from typing import Dict
from typing import List
from typing import Optional
from typing import Tuple
from typing import Union

from numbers import Number
import logging as log

# Load dataset

In [3]:
dicts = LoadDicts("../../random-forest-mc/tests/")
dataset_dict = dicts.datasets_metadata
ds_name = "titanic"
params = dataset_dict[ds_name]
dataset = (
    pd.read_csv(params["csv_path"])[params["ds_cols"] + [params["target_col"]]]
    .dropna()
    .reset_index(drop=True)
)
dataset["Age"] = dataset["Age"].astype(np.uint8)
dataset["SibSp"] = dataset["SibSp"].astype(np.uint8)
dataset["Pclass"] = dataset["Pclass"].astype(str)
dataset["Fare"] = dataset["Fare"].astype(np.uint32)
ds_cols = params["ds_cols"]
target_col = params["target_col"]

# Load model

In [4]:
cls = RandomForestMC(target_col=target_col)
cls.fit(dataset)
row = dataset.reset_index(drop=True).loc[0]
predict_row = cls.predict(row)
predict_ds = cls.predict(dataset.sample(n=10))
predict_probs_ds = cls.predict_proba(dataset.sample(n=10))

Planting the forest: 100%|██████████| 16/16 [00:06<00:00,  2.46it/s]


In [5]:
predict_ds

['1', '0', '0', '1', '1', '0', '1', '0', '0', '1']

In [6]:
cls

RandomForestMC(len(Forest)=16,n_trees=16,model_version=1.1.0,module_version=1.1.0)

# Generate some missing data

In [7]:
df_tmp = dataset.sample(frac=0.2).reset_index(drop=True)
mask_random = np.random.choice([True, False], size=df_tmp[ds_cols].shape, p=[0.7, 0.3])
dataset_missing_values = df_tmp[ds_cols].mask(~mask_random)
dataset_missing_values[target_col] = df_tmp[target_col]
dataset_missing_values_ground_truth = df_tmp

In [8]:
dataset_missing_values

,Pclass,Sex,Age,SibSp,Fare,Embarked,Survived
0,3,male,NaN,0.0,8.0,S,0
1,NaN,male,36.0,NaN,15.0,S,0
2,NaN,male,21.0,0.0,NaN,Q,0
3,1,female,2.0,1.0,NaN,S,0
4,1,NaN,64.0,1.0,263.0,S,0
...,...,...,...,...,...,...,...
137,NaN,female,18.0,0.0,79.0,NaN,1
138,3,NaN,34.0,0.0,8.0,NaN,0
139,1,NaN,54.0,0.0,77.0,S,0
140,2,NaN,37.0,NaN,26.0,NaN,0


In [9]:
dict_values = {col: dataset[col].unique().tolist() for col in ds_cols}

In [10]:
dict_values.keys()

dict_keys(['Pclass', 'Sex', 'Age', 'SibSp', 'Fare', 'Embarked'])

In [26]:
for i, row in dataset_missing_values.iterrows():
    if row.isna().any():
        break
df_pred_missings = cls.predictMissingValues(row, dict_values)
df_pred_missings

,Pclass,Sex,Age,SibSp,Fare,Embarked,Survived,0,1,row_id
0,3,male,NaN,0.0,8.0,S,0,NaN,NaN,0
1,3,male,22,0.0,8.0,S,0,0.8750,0.1250,0
2,3,male,38,0.0,8.0,S,0,0.9375,0.0625,0
3,3,male,26,0.0,8.0,S,0,0.8750,0.1250,0
4,3,male,35,0.0,8.0,S,0,0.9375,0.0625,0
...,...,...,...,...,...,...,...,...,...,...
67,3,male,53,0.0,8.0,S,0,0.9375,0.0625,0
68,3,male,57,0.0,8.0,S,0,0.9375,0.0625,0
69,3,male,80,0.0,8.0,S,0,0.9375,0.0625,0
70,3,male,6,0.0,8.0,S,0,0.7500,0.2500,0


In [13]:
df_pred_missings = cls.predictMissingValues(dataset_missing_values.sample(n=2), dict_values)
df_pred_missings

,Pclass,Sex,Age,SibSp,Fare,Embarked,Survived,0,1,row_id
0,NaN,male,21.0,0.0,NaN,Q,0,NaN,NaN,0
1,1,NaN,27.0,0.0,30.0,S,1,NaN,NaN,1
2,1,male,27.0,0.0,30.0,S,1,0.5625,0.4375,1
3,1,female,27.0,0.0,30.0,S,1,0.0625,0.9375,1


In [14]:
dataset_missing_values

,Pclass,Sex,Age,SibSp,Fare,Embarked,Survived
0,3,male,NaN,0.0,8.0,S,0
1,NaN,male,36.0,NaN,15.0,S,0
2,NaN,male,21.0,0.0,NaN,Q,0
3,1,female,2.0,1.0,NaN,S,0
4,1,NaN,64.0,1.0,263.0,S,0
...,...,...,...,...,...,...,...
137,NaN,female,18.0,0.0,79.0,NaN,1
138,3,NaN,34.0,0.0,8.0,NaN,0
139,1,NaN,54.0,0.0,77.0,S,0
140,2,NaN,37.0,NaN,26.0,NaN,0


In [30]:
df_tmp.isna().any().any()

True

In [31]:
while True:
    df_tmp = dataset_missing_values.sample(n=20)
    if df_tmp.isna().any().any():
        break
df_pred_missings = cls.predictMissingValues(df_tmp, dict_values)
df_pred_missings

,Pclass,Sex,Age,SibSp,Fare,Embarked,Survived,0,1,row_id
0,NaN,NaN,NaN,0.0,NaN,S,0,NaN,NaN,0
1,NaN,female,27.0,0.0,10.0,S,1,NaN,NaN,1
2,NaN,male,21.0,0.0,NaN,Q,0,NaN,NaN,2
3,3,male,20.0,0.0,7.0,NaN,0,NaN,NaN,3
4,3,male,20.0,0.0,7.0,S,0,0.8750,0.1250,3
...,...,...,...,...,...,...,...,...,...,...
472,2,male,48.0,0.0,5,NaN,0,0.8125,0.1875,18
473,2,male,48.0,0.0,NaN,S,0,0.8750,0.1250,18
474,2,male,48.0,0.0,NaN,C,0,0.8125,0.1875,18
475,2,male,48.0,0.0,NaN,Q,0,0.8125,0.1875,18


In [16]:
df_pred_missings = cls.predictMissingValues(dataset.sample(n=20), dict_values)
df_pred_missings

MissingValuesNotFound: Dataset or row without missing values! Please, give a dataset or row with missing values using 'NaN'.

In [ ]:
df_pred_missings = cls.predictMissingValues(dataset.loc[0], dict_values)
df_pred_missings

# Example for README.md

In [43]:
df_tmp = dataset.sample(frac=0.2).reset_index(drop=True)
mask_random = np.random.choice([True, False], size=df_tmp[ds_cols].shape, p=[0.9, 0.1])
dataset_missing_values = df_tmp[ds_cols].mask(~mask_random)
dataset_missing_values[target_col] = df_tmp[target_col]
#dataset_missing_values_ground_truth = df_tmp

In [51]:
dataset_missing_values = dataset_missing_values.loc[dataset_missing_values['Embarked'].isna() | dataset_missing_values['Sex'].isna()].head(2).reset_index(drop=True)
dataset_missing_values

,Pclass,Sex,Age,SibSp,Fare,Embarked,Survived
0,1,NaN,45.0,0.0,28.0,S,0
1,1,male,34.0,0.0,26.0,NaN,1


In [53]:
print(dataset_missing_values.to_markdown())

|    |   Pclass | Sex   |   Age |   SibSp |   Fare | Embarked   |   Survived |
|---:|---------:|:------|------:|--------:|-------:|:-----------|-----------:|
|  0 |        1 | nan   |    45 |       0 |     28 | S          |          0 |
|  1 |        1 | male  |    34 |       0 |     26 | nan        |          1 |


In [42]:
dict_values

{'Pclass': ['3', '1', '2'],
 'Sex': ['male', 'female'],
 'Age': [22,
  38,
  26,
  35,
  54,
  2,
  27,
  14,
  4,
  58,
  20,
  39,
  55,
  31,
  34,
  15,
  28,
  8,
  19,
  40,
  66,
  42,
  21,
  18,
  3,
  7,
  49,
  29,
  65,
  5,
  11,
  45,
  17,
  32,
  16,
  25,
  0,
  30,
  33,
  23,
  24,
  46,
  59,
  71,
  37,
  47,
  70,
  12,
  9,
  36,
  51,
  44,
  1,
  61,
  56,
  50,
  62,
  41,
  52,
  63,
  43,
  60,
  10,
  64,
  13,
  48,
  53,
  57,
  80,
  6,
  74],
 'SibSp': [1, 0, 3, 4, 2, 5],
 'Fare': [7,
  71,
  53,
  8,
  51,
  21,
  11,
  30,
  16,
  26,
  31,
  29,
  18,
  13,
  35,
  263,
  27,
  10,
  82,
  52,
  9,
  41,
  17,
  39,
  76,
  61,
  46,
  83,
  73,
  14,
  56,
  12,
  47,
  15,
  34,
  20,
  63,
  23,
  77,
  247,
  6,
  79,
  36,
  66,
  33,
  28,
  0,
  22,
  146,
  113,
  90,
  86,
  512,
  153,
  135,
  19,
  78,
  91,
  151,
  108,
  24,
  262,
  164,
  134,
  57,
  55,
  75,
  69,
  211,
  4,
  227,
  120,
  32,
  81,
  89,
  38,
  49,
  59,
  93,

In [41]:
print(dataset_missing_values.head(2).to_markdown())

|    |   Pclass | Sex   |   Age |   SibSp |   Fare | Embarked   |   Survived |
|---:|---------:|:------|------:|--------:|-------:|:-----------|-----------:|
|  0 |        1 | male  |    64 |       0 |     26 | nan        |          0 |
|  1 |        3 | male  |    26 |       0 |    nan | S          |          0 |


In [56]:
df_tmp = cls.predictMissingValues(dataset_missing_values, dict_values)
df_tmp

,Pclass,Sex,Age,SibSp,Fare,Embarked,Survived,0,1,row_id
0,1,NaN,45.0,0.0,28.0,S,0,NaN,NaN,0
1,1,male,45.0,0.0,28.0,S,0,0.5625,0.4375,0
2,1,female,45.0,0.0,28.0,S,0,0.0625,0.9375,0
3,1,male,34.0,0.0,26.0,NaN,1,NaN,NaN,1
4,1,male,34.0,0.0,26.0,S,1,0.5625,0.4375,1
5,1,male,34.0,0.0,26.0,C,1,0.5625,0.4375,1
6,1,male,34.0,0.0,26.0,Q,1,0.5625,0.4375,1


In [58]:
print(df_tmp.to_markdown())

|    |   Pclass | Sex    |   Age |   SibSp |   Fare | Embarked   |   Survived |        0 |        1 |   row_id |
|---:|---------:|:-------|------:|--------:|-------:|:-----------|-----------:|---------:|---------:|---------:|
|  0 |        1 | nan    |    45 |       0 |     28 | S          |          0 | nan      | nan      |        0 |
|  1 |        1 | male   |    45 |       0 |     28 | S          |          0 |   0.5625 |   0.4375 |        0 |
|  2 |        1 | female |    45 |       0 |     28 | S          |          0 |   0.0625 |   0.9375 |        0 |
|  3 |        1 | male   |    34 |       0 |     26 | nan        |          1 | nan      | nan      |        1 |
|  4 |        1 | male   |    34 |       0 |     26 | S          |          1 |   0.5625 |   0.4375 |        1 |
|  5 |        1 | male   |    34 |       0 |     26 | C          |          1 |   0.5625 |   0.4375 |        1 |
|  6 |        1 | male   |    34 |       0 |     26 | Q          |          1 |   0.5625 |   0.4